In [222]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import h5py
from PIL import Image
from src.benchmarkers import *
from src.benchmarkersV2 import *
from tqdm import tqdm
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from io import BytesIO

np.random.seed(0)

In [231]:
def area(ctx,array):
    format = pc.list_element(array, 4)
    result = []
    for i in range(40):
        if i == 0 :
            mask = pc.equal(format,i)
            result.append(pc.if_else(mask,pc.multiply(pc.list_element(array,2),pc.list_element(array,3)),None))
        elif i == 1 :
            mask = pc.equal(format,i)
            result.append(pc.if_else(mask,pc.multiply(pc.subtract(pc.list_element(array,3),pc.list_element(array,1)),pc.subtract(pc.list_element(array,2),pc.list_element(array,0))),None))

    final_result = 0
    for i, item in enumerate(result):
        if i+1 < len(result):
            final_result = item.fill_null(result[i+1])
    
    return final_result

In [224]:
table = pq.read_table(r"C:\Users\Cristiano Lavoro\Downloads\ds_10.parquet")

In [230]:
test_array = table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("bbox")

<pyarrow.ListScalar: [0.5, 0.5, 0.5, 0.5, 0.0]>

In [226]:
test_array = table.column("image_feature").chunk(0)[0].values[0]["boundingbox_feature"]
print(len(test_array))
print(type(test_array))
test_array

3
<class 'pyarrow.lib.ListScalar'>


<pyarrow.ListScalar: [{'bbox': [0.5, 0.5, 0.5, 0.5, 0.0], 'image_1_feature': [{'image': b'\x00H\xc2\x94\xda\xc6d?\x91\x98@8\xb6\xa9\xe8?\xe1\xc9\x13\x14G\xcf\xec?4p\xcef\x87-\xd0?\x08E5\xea:\r\xc1?:\xcf\xa1\x9fE\x97\xe8?8\x97\x0c\x88m\\\xe7?\xc2sX\xc3\xa7`\xe9?ig\x134\xfdg\xed?pM\x90\xbb\xba\xa6\xa6?\xbd\x1b\x97O`/\xe1?,\x8a\xd9\xd7\x17@\xe9?\xf5\xb6P\xf0\x1b\n\xe0?\xe9\xcf\x89N&\xab\xe1?\xf1\x90\xe7\xd0\xc6!\xe2?@\x96\x848\x1eG\x85?\x1bSN\x82\xee\xef\xec?X\x8c\xc9/\x17\xfb\xca? i>\x05\xb0\x11\x9a?\xc8\x07|\xb8!E\xe4?\xbe0\xd8\x02\xf2\xec\xe1?\xa4\x863qjZ\xc0?\xd3\xd7\n\xb8\x07\xf2\xef?\x18%\xe3\xd6@\x0e\xd9?\xbc6\xb3.\x0c\x06\xd1?HT\x96O\n\xe8\xb9?+\xe7h\xed\xed\x1a\xe1?\x04Y4-\xfc\x06\xc5?X\xc2,\x08~\xa5\xc2?\xbfI\xa7+\xbfG\xe7?\xa0\x85\xf4\xfb\xc7\x1d\xb1?\x0eZ;\xb8#\xd2\xe6?p\xaa\xe3{E\xd4\xb1?\x1e\xfb\xfc\xfcn<\xd9?\xa8\xa2\x96s\xf8\xfb\xbf?\'\x81\x06\xfaR\x82\xef?\xfc\xd6\xcbqL=\xce?\xb2\xeb\xb3\\YS\xd7?0\xaaFz\xe6\x0c\xcb?\xdb\x97.\x89\x7fg\xe1?g\x05\x82\x0e9\x83\xed?\x86\x1b\xe

In [233]:
import pyarrow as pa
import pyarrow.compute as pc


func_doc = {}
func_doc["summary"] = "simple bbox"
func_doc["description"] = "calculate bbox area"
func_name = "area2"
in_types = {"array": pa.list_(pa.float64())}
out_type = pa.float64()

pc.register_scalar_function(area, func_name, func_doc, in_types, out_type)
func = pc.get_function(func_name)

answer = pc.call_function(func_name, [test_array])


In [234]:
answer

[
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  ...
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999,
  0.25,
  0.25,
  0.25,
  0.4068489599999999,
  0.4068489599999999
]